## SOAR Data
This Jupyter notebook imports data about the SOAR facility.

There are two parts to this notebook.
1. Some simple code to read and decode the SOAR data (in JSON format);
2. Code that makes use of the VeRoViz package to visualize the SOAR data.

--- 

### 1.  Read JSON data

In [1]:
import json
import urllib.request

In [15]:
with urllib.request.urlopen("https://raw.githubusercontent.com/optimatorlab/SOAR/master/data/soar.json") as url:
    soar_data = json.loads(url.read().decode())

soar_data

{'polesArray': [{'id': 0,
   'lat': 42.995254771,
   'lon': -78.796842098,
   'elevMSLmeters': 180.43,
   'heightMeters': 25.908},
  {'id': 1,
   'lat': 42.995348937,
   'lon': -78.796745539,
   'elevMSLmeters': 180.61,
   'heightMeters': 25.908},
  {'id': 2,
   'lat': 42.99543918,
   'lon': -78.796648979,
   'elevMSLmeters': 180.69,
   'heightMeters': 25.908},
  {'id': 3,
   'lat': 42.995505881,
   'lon': -78.796557784,
   'elevMSLmeters': 180.86,
   'heightMeters': 25.908},
  {'id': 4,
   'lat': 42.995588276,
   'lon': -78.796713352,
   'elevMSLmeters': 181.16,
   'heightMeters': 25.908},
  {'id': 5,
   'lat': 42.995674594,
   'lon': -78.796863556,
   'elevMSLmeters': 181.33,
   'heightMeters': 25.908},
  {'id': 6,
   'lat': 42.995760912,
   'lon': -78.797008395,
   'elevMSLmeters': 181.45,
   'heightMeters': 25.908},
  {'id': 7,
   'lat': 42.995859001,
   'lon': -78.797163963,
   'elevMSLmeters': 181.55,
   'heightMeters': 25.908},
  {'id': 8,
   'lat': 42.995776607,
   'lon': -78.7

In [14]:
for item in soar_data['polesArray']:
    print('id:', item['id'], 
          'lat:', item['lat'], 
          'lon:', item['lon'],
          'elevMSLmeters:', item['elevMSLmeters'],
          'heightMeters:', item['heightMeters'])

id: 0 lat: 42.995254771 lon: -78.796842098 elevMSLmeters: 180.43 heightMeters: 25.908
id: 1 lat: 42.995348937 lon: -78.796745539 elevMSLmeters: 180.61 heightMeters: 25.908
id: 2 lat: 42.99543918 lon: -78.796648979 elevMSLmeters: 180.69 heightMeters: 25.908
id: 3 lat: 42.995505881 lon: -78.796557784 elevMSLmeters: 180.86 heightMeters: 25.908
id: 4 lat: 42.995588276 lon: -78.796713352 elevMSLmeters: 181.16 heightMeters: 25.908
id: 5 lat: 42.995674594 lon: -78.796863556 elevMSLmeters: 181.33 heightMeters: 25.908
id: 6 lat: 42.995760912 lon: -78.797008395 elevMSLmeters: 181.45 heightMeters: 25.908
id: 7 lat: 42.995859001 lon: -78.797163963 elevMSLmeters: 181.55 heightMeters: 25.908
id: 8 lat: 42.995776607 lon: -78.797249794 elevMSLmeters: 181.44 heightMeters: 25.908
id: 9 lat: 42.995682441 lon: -78.797335625 elevMSLmeters: 181.39 heightMeters: 25.908
id: 10 lat: 42.995588276 lon: -78.797437549 elevMSLmeters: 181.36 heightMeters: 25.908
id: 11 lat: 42.995517651 lon: -78.797298074 elevMSLmet

### 2. Visualize data on maps 

You'll need the `veroviz` package to continue.  To install:
```
pip install veroviz
```

In [16]:
import veroviz as vrv
vrv.checkVersion()

'Your current installed version of veroviz is 0.4.5. You are up-to-date with the latest available version.'

In [32]:
# Create a "nodes" dataframe from the `soar_data` object (imported in Step 1)
myNodes = vrv.initDataframe('nodes')

for item in soar_data['polesArray']:
    locs = [item['lat'], item['lon']]
    myNodes = vrv.createNodesFromLocs(
                    locs              = [locs], 
                    initNodes         = myNodes,
                    startNode         = item['id'],
                    nodeType          = 'Poles', 
                    nodeName          = 'pole%d' % item['id'], 
                    leafletIconPrefix = 'custom', 
                    leafletIconType   = '10-white-10', 
                    leafletIconText   = item['id'])

In [62]:
# Initialize map and draw blue poles
myMap = vrv.createLeaflet(nodes = myNodes, mapBackground = 'Arcgis Aerial', zoomStart = 18)
myMap

In [63]:
# Add curbs
for i in soar_data['curbsArray']:
    myMap = vrv.addLeafletPolygon(mapObject = myMap,
                              zoomStart     = 18,
                              points        = i['poly'], 
                              lineWeight    = 3, 
                              lineColor     = 'pink', 
                              lineOpacity   = 0.8, 
                              lineStyle     = 'solid',
                              fillColor     = 'pink', 
                              fillOpacity   = 0.5)
myMap

In [57]:
# Add light poles
for i in soar_data['lightsArray']:
    myMap = vrv.addLeafletCircle(mapObject   = myMap, 
                                 zoomStart   = 18,
                                 center      = [i['lat'], i['lon']], 
                                 radius      = i['radius'], 
                                 popupText   = 'lightpole %d' % i['id'], 
                                 lineWeight  = 3, 
                                 lineColor   = None, 
                                 lineOpacity = 0.8, 
                                 lineStyle   = 'solid', 
                                 fillColor   = 'red', 
                                 fillOpacity = 0.8)
myMap

In [64]:
# Add geofence
myMap = vrv.addLeafletPolygon(mapObject=myMap, 
                              zoomStart   = 18, 
                              points      = soar_data['geofence']['poly'], 
                              popupText   = None, 
                              lineWeight  = 3, 
                              lineColor   = 'red', 
                              lineOpacity = 0.8, 
                              lineStyle   = 'solid', 
                              fillColor   = None, 
                              fillOpacity = 0.3)
myMap

--- 

### 3.  Error-Checking

We need to bet better readings on our poles!

In [92]:
# Poles 0-1, 1-2, 2-3, 7-8, 8-9, 9-10 should be 40-feet apart.
print('Poles 0-1, 1-2, 2-3, 7-8, 8-9, 9-10 should be 40-feet apart.')
for (i,j) in [[0,1], [1,2], [2,3], [7,8], [8,9], [9,10]]:
    loc1 = list(myNodes[myNodes['id'] == i][['lat', 'lon']].values[0])
    loc2 = list(myNodes[myNodes['id'] == j][['lat', 'lon']].values[0])
    distMeters = vrv.distance2D(loc1, loc2)
    distFeet = vrv.convertDistance(distMeters, 'meters', 'feet')
    print('From Pole %d to %d: %f meters (%f feet)' % (i, j, distMeters, distFeet))


# Poles 3-4, 4-5, 5-6, 6-7, 10-11, 11-12, 12-13, 13-0 should be 50-feet apart.
print('\n\nPoles 3-4, 4-5, 5-6, 6-7, 10-11, 11-12, 12-13, 13-0 should be 50-feet apart.')
for (i,j) in [[3,4], [4,5], [5,6], [6,7], [10,11], [11,12], [12,13], [13,0]]:
    loc1 = list(myNodes[myNodes['id'] == i][['lat', 'lon']].values[0])
    loc2 = list(myNodes[myNodes['id'] == j][['lat', 'lon']].values[0])
    distMeters = vrv.distance2D(loc1, loc2)
    distFeet = vrv.convertDistance(distMeters, 'meters', 'feet')
    print('From Pole %d to %d: %f meters (%f feet)' % (i, j, distMeters, distFeet))

Poles 0-1, 1-2, 2-3, 7-8, 8-9, 9-10 should be 40-feet apart.
From Pole 0 to 1: 13.093408 meters (42.957375 feet)
From Pole 1 to 2: 12.747944 meters (41.823965 feet)
From Pole 2 to 3: 10.498195 meters (34.442896 feet)
From Pole 7 to 8: 11.522725 meters (37.804216 feet)
From Pole 8 to 9: 12.586694 meters (41.294927 feet)
From Pole 9 to 10: 13.360971 meters (43.835206 feet)


Poles 3-4, 4-5, 5-6, 6-7, 10-11, 11-12, 12-13, 13-0 should be 50-feet apart.
From Pole 3 to 4: 15.643620 meters (51.324211 feet)
From Pole 4 to 5: 15.555833 meters (51.036198 feet)
From Pole 5 to 6: 15.213725 meters (49.913797 feet)
From Pole 6 to 7: 16.723612 meters (54.867492 feet)
From Pole 10 to 11: 13.817400 meters (45.332678 feet)
From Pole 11 to 12: 16.394329 meters (53.787170 feet)
From Pole 12 to 13: 14.425377 meters (47.327351 feet)
From Pole 13 to 0: 16.514752 meters (54.182259 feet)
